# A3 Statistical Analysis: Bootstrap CI & Supplementary Metrics

**Purpose**: 
1. Calculate 95% confidence intervals for λ* using Bootstrap
2. Compute supplementary metrics (Δ@λ=1, harm area)
3. Perform McNemar test for significance
4. Create publication-quality figures

**Input**: cot_results_*.json and direct_results_*.json from all models

## 0. Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import brentq
from scipy import stats
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Base directory
BASE_DIR = '/content/drive/MyDrive/CoT_Experiment'

# Output directory for analysis
ANALYSIS_DIR = f'{BASE_DIR}/a3_statistical_analysis'
os.makedirs(ANALYSIS_DIR, exist_ok=True)
os.makedirs(f'{ANALYSIS_DIR}/figures', exist_ok=True)

print(f'Analysis output: {ANALYSIS_DIR}')

## 1. Load All Experimental Data

In [ ]:
def load_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

def save_json(data, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# Model configurations - UPDATE PATHS AS NEEDED
MODEL_CONFIGS = {
    'Claude 3 Haiku': {
        'dir': 'a3_claude_haiku3_20251228',
        'short': 'haiku',
        'color': '#1f77b4',
        'marker': 'o'
    },
    'Claude 3.5 Haiku': {
        'dir': 'a3_claude_haiku35_20251228',
        'short': 'haiku35',
        'color': '#2ca02c',
        'marker': 's'
    },
    'Claude 4 Sonnet': {
        'dir': 'a3_claude_sonnet4_20251228',
        'short': 'sonnet4',
        'color': '#9467bd',
        'marker': '^'
    },
    'GPT-3.5 Turbo': {
        'dir': 'a3_gpt_gpt35_20251228',
        'short': 'gpt35',
        'color': '#d62728',
        'marker': 'v'
    },
    'GPT-4o-mini': {
        'dir': 'a3_gpt_gpt4omini_20251228',
        'short': 'gpt4omini',
        'color': '#ff7f0e',
        'marker': 'D'
    },
    'GPT-4o': {
        'dir': 'chatgpt_experiment_20251225',  # Original experiment
        'short': 'gpt4o',
        'color': '#8c564b',
        'marker': 'p',
        'alt_dir': 'a3_gpt_gpt4o_20251228'  # Alternative if exists
    }
}

# Load data for each model
all_data = {}

for model_name, config in MODEL_CONFIGS.items():
    print(f'\nLoading {model_name}...')
    
    # Try primary directory
    model_dir = f"{BASE_DIR}/{config['dir']}"
    
    # Check alternative directory if primary doesn't exist
    if not os.path.exists(model_dir) and 'alt_dir' in config:
        model_dir = f"{BASE_DIR}/{config['alt_dir']}"
    
    if not os.path.exists(model_dir):
        print(f'  ⚠️ Directory not found: {model_dir}')
        continue
    
    try:
        # Load direct results
        direct_path = f"{model_dir}/results/direct_results_{config['short']}.json"
        if not os.path.exists(direct_path):
            # Try alternative naming
            for f in os.listdir(f"{model_dir}/results"):
                if f.startswith('direct_results'):
                    direct_path = f"{model_dir}/results/{f}"
                    break
        
        direct_results = load_json(direct_path)
        
        # Load CoT results
        cot_path = f"{model_dir}/results/cot_results_{config['short']}.json"
        if not os.path.exists(cot_path):
            for f in os.listdir(f"{model_dir}/results"):
                if f.startswith('cot_results'):
                    cot_path = f"{model_dir}/results/{f}"
                    break
        
        cot_results = load_json(cot_path)
        
        all_data[model_name] = {
            'direct': direct_results,
            'cot': cot_results,
            'config': config
        }
        
        print(f'  ✓ Loaded: {len(direct_results)} direct, {len(cot_results)} cot')
        
    except Exception as e:
        print(f'  ❌ Error: {e}')

print(f'\n=== Loaded {len(all_data)} models ===')

## 2. Compute Basic Metrics for Each Model

In [ ]:
def compute_basic_metrics(direct_results, cot_results):
    """Compute baseline accuracy and accuracy by lambda"""
    
    # Baseline accuracy
    baseline = sum(r['is_correct'] for r in direct_results) / len(direct_results)
    
    # Accuracy by lambda
    cot_df = pd.DataFrame(cot_results)
    acc_by_lam = cot_df.groupby('lam')['is_correct'].mean().to_dict()
    
    return baseline, acc_by_lam

def estimate_lambda_crit(lam_arr, acc_arr, baseline):
    """Find λ where CoT accuracy crosses baseline"""
    lam_arr = np.array(lam_arr)
    acc_arr = np.array(acc_arr)
    
    try:
        f = interp1d(lam_arr, acc_arr - baseline, kind='linear', fill_value='extrapolate')
        for i in range(len(lam_arr) - 1):
            if (acc_arr[i] - baseline) * (acc_arr[i+1] - baseline) < 0:
                return brentq(f, lam_arr[i], lam_arr[i+1])
    except:
        pass
    
    # If no crossing found
    if acc_arr[-1] > baseline:
        return 1.0  # Never crosses
    return None

# Compute metrics for all models
metrics = {}

for model_name, data in all_data.items():
    baseline, acc_by_lam = compute_basic_metrics(data['direct'], data['cot'])
    
    lam_arr = sorted(acc_by_lam.keys())
    acc_arr = [acc_by_lam[l] for l in lam_arr]
    
    lambda_crit = estimate_lambda_crit(lam_arr, acc_arr, baseline)
    
    metrics[model_name] = {
        'baseline': baseline,
        'acc_by_lam': acc_by_lam,
        'lambda_crit': lambda_crit,
        'delta_at_1': acc_by_lam.get(1.0, 0) - baseline,  # Δ@λ=1
        'n_problems': len(data['direct'])
    }

# Display
print('='*70)
print('BASIC METRICS')
print('='*70)
print(f'{"Model":<20} {"Baseline":>10} {"λ*":>10} {"Δ@λ=1":>10} {"N":>6}')
print('-'*70)
for model, m in sorted(metrics.items(), key=lambda x: x[1]['baseline']):
    lc = f"{m['lambda_crit']:.3f}" if m['lambda_crit'] else 'N/A'
    print(f"{model:<20} {m['baseline']:>10.1%} {lc:>10} {m['delta_at_1']:>+10.1%} {m['n_problems']:>6}")

## 3. Bootstrap Analysis for λ* Confidence Intervals

In [ ]:
def bootstrap_lambda_crit(direct_results, cot_results, n_bootstrap=1000, seed=42):
    """
    Bootstrap estimation of λ* with 95% CI
    
    Returns: (point_estimate, ci_lower, ci_upper, bootstrap_samples)
    """
    np.random.seed(seed)
    
    # Get problem indices
    direct_df = pd.DataFrame(direct_results)
    cot_df = pd.DataFrame(cot_results)
    
    problem_ids = direct_df['problem_index'].unique()
    n_problems = len(problem_ids)
    
    # Point estimate
    baseline = direct_df['is_correct'].mean()
    acc_by_lam = cot_df.groupby('lam')['is_correct'].mean()
    lam_arr = np.array(acc_by_lam.index)
    acc_arr = np.array(acc_by_lam.values)
    point_estimate = estimate_lambda_crit(lam_arr, acc_arr, baseline)
    
    # Bootstrap
    bootstrap_samples = []
    
    for b in range(n_bootstrap):
        # Resample problem indices with replacement
        boot_ids = np.random.choice(problem_ids, size=n_problems, replace=True)
        
        # Get bootstrap sample for direct
        boot_direct = direct_df[direct_df['problem_index'].isin(boot_ids)]
        boot_baseline = boot_direct['is_correct'].mean()
        
        # Get bootstrap sample for CoT
        boot_cot = cot_df[cot_df['problem_index'].isin(boot_ids)]
        boot_acc = boot_cot.groupby('lam')['is_correct'].mean()
        
        if len(boot_acc) == len(lam_arr):
            boot_lc = estimate_lambda_crit(
                np.array(boot_acc.index),
                np.array(boot_acc.values),
                boot_baseline
            )
            if boot_lc is not None:
                bootstrap_samples.append(boot_lc)
    
    if len(bootstrap_samples) > 0:
        ci_lower = np.percentile(bootstrap_samples, 2.5)
        ci_upper = np.percentile(bootstrap_samples, 97.5)
        boot_mean = np.mean(bootstrap_samples)
        boot_std = np.std(bootstrap_samples)
    else:
        ci_lower, ci_upper, boot_mean, boot_std = None, None, None, None
    
    return {
        'point_estimate': point_estimate,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'boot_mean': boot_mean,
        'boot_std': boot_std,
        'n_valid_samples': len(bootstrap_samples),
        'samples': bootstrap_samples
    }

# Run bootstrap for all models
print('Running Bootstrap analysis (this may take a few minutes)...')
print('='*70)

bootstrap_results = {}

for model_name, data in tqdm(all_data.items(), desc='Bootstrap'):
    result = bootstrap_lambda_crit(
        data['direct'], 
        data['cot'], 
        n_bootstrap=1000
    )
    bootstrap_results[model_name] = result
    
    pe = result['point_estimate']
    ci_l = result['ci_lower']
    ci_u = result['ci_upper']
    
    if pe and ci_l and ci_u:
        print(f"{model_name}: λ* = {pe:.3f} [95% CI: {ci_l:.3f}, {ci_u:.3f}]")
    else:
        print(f"{model_name}: λ* = {pe} (CI could not be computed)")

## 4. McNemar Test: Is Backfire Significant?

In [ ]:
def mcnemar_test(direct_results, cot_results, target_lambda):
    """
    McNemar test comparing Direct vs CoT at specific λ
    
    Tests if the difference in accuracy is statistically significant
    """
    direct_df = pd.DataFrame(direct_results)
    cot_df = pd.DataFrame(cot_results)
    
    # Filter CoT for target lambda
    cot_at_lam = cot_df[cot_df['lam'] == target_lambda]
    
    # Merge on problem_index
    merged = pd.merge(
        direct_df[['problem_index', 'is_correct']],
        cot_at_lam[['problem_index', 'is_correct']],
        on='problem_index',
        suffixes=('_direct', '_cot')
    )
    
    # Build contingency table
    # a: both correct, b: direct correct/cot wrong
    # c: direct wrong/cot correct, d: both wrong
    a = ((merged['is_correct_direct'] == True) & (merged['is_correct_cot'] == True)).sum()
    b = ((merged['is_correct_direct'] == True) & (merged['is_correct_cot'] == False)).sum()
    c = ((merged['is_correct_direct'] == False) & (merged['is_correct_cot'] == True)).sum()
    d = ((merged['is_correct_direct'] == False) & (merged['is_correct_cot'] == False)).sum()
    
    # McNemar statistic (with continuity correction)
    if b + c > 0:
        chi2 = (abs(b - c) - 1)**2 / (b + c)
        p_value = 1 - stats.chi2.cdf(chi2, df=1)
    else:
        chi2, p_value = 0, 1.0
    
    return {
        'lambda': target_lambda,
        'contingency': [[a, b], [c, d]],
        'direct_correct_cot_wrong': b,
        'direct_wrong_cot_correct': c,
        'chi2': chi2,
        'p_value': p_value,
        'significant': p_value < 0.05
    }

# Run McNemar test for key lambda values
print('='*70)
print('McNEMAR TEST: Is Backfire Statistically Significant?')
print('='*70)

mcnemar_results = {}

for model_name, data in all_data.items():
    print(f'\n{model_name}:')
    model_results = {}
    
    for lam in [0.4, 0.6, 0.8, 1.0]:
        result = mcnemar_test(data['direct'], data['cot'], lam)
        model_results[lam] = result
        
        sig = '***' if result['p_value'] < 0.001 else '**' if result['p_value'] < 0.01 else '*' if result['p_value'] < 0.05 else ''
        direction = 'CoT better' if result['direct_wrong_cot_correct'] > result['direct_correct_cot_wrong'] else 'Direct better'
        
        print(f"  λ={lam}: p={result['p_value']:.4f} {sig} ({direction})")
    
    mcnemar_results[model_name] = model_results

## 5. Compute Supplementary Metrics

In [ ]:
def compute_harm_area(acc_by_lam, baseline):
    """
    Compute the "harm area" - integral of (baseline - acc) where acc < baseline
    Uses trapezoidal integration
    """
    lams = sorted(acc_by_lam.keys())
    harm = 0
    
    for i in range(len(lams) - 1):
        l1, l2 = lams[i], lams[i+1]
        a1, a2 = acc_by_lam[l1], acc_by_lam[l2]
        
        # Compute harm (negative benefit) for this segment
        h1 = max(0, baseline - a1)
        h2 = max(0, baseline - a2)
        
        # Trapezoidal area
        harm += (h1 + h2) * (l2 - l1) / 2
    
    return harm

def compute_benefit_area(acc_by_lam, baseline):
    """
    Compute the "benefit area" - integral of (acc - baseline) where acc > baseline
    """
    lams = sorted(acc_by_lam.keys())
    benefit = 0
    
    for i in range(len(lams) - 1):
        l1, l2 = lams[i], lams[i+1]
        a1, a2 = acc_by_lam[l1], acc_by_lam[l2]
        
        b1 = max(0, a1 - baseline)
        b2 = max(0, a2 - baseline)
        
        benefit += (b1 + b2) * (l2 - l1) / 2
    
    return benefit

# Compute all supplementary metrics
print('='*70)
print('SUPPLEMENTARY METRICS')
print('='*70)

supplementary = {}

for model_name, m in metrics.items():
    harm = compute_harm_area(m['acc_by_lam'], m['baseline'])
    benefit = compute_benefit_area(m['acc_by_lam'], m['baseline'])
    
    supplementary[model_name] = {
        'lambda_crit': m['lambda_crit'],
        'delta_at_1': m['delta_at_1'],
        'harm_area': harm,
        'benefit_area': benefit,
        'net_area': benefit - harm,
        'harm_ratio': harm / (harm + benefit) if (harm + benefit) > 0 else 0
    }

print(f'\n{"Model":<20} {"λ*":>8} {"Δ@λ=1":>10} {"Harm":>8} {"Benefit":>8} {"Net":>8}')
print('-'*70)
for model, s in sorted(supplementary.items(), key=lambda x: metrics[x[0]]['baseline']):
    lc = f"{s['lambda_crit']:.3f}" if s['lambda_crit'] else 'N/A'
    print(f"{model:<20} {lc:>8} {s['delta_at_1']:>+10.3f} {s['harm_area']:>8.3f} {s['benefit_area']:>8.3f} {s['net_area']:>+8.3f}")

## 6. Publication-Quality Figures

In [ ]:
# Figure 1: Baseline vs λ* (Main Result)

fig, ax = plt.subplots(figsize=(10, 8))

for model_name, data in all_data.items():
    m = metrics[model_name]
    b = bootstrap_results[model_name]
    config = data['config']
    
    if m['lambda_crit'] is not None:
        # Plot point
        ax.scatter(
            m['baseline'] * 100, 
            m['lambda_crit'],
            s=200, 
            c=config['color'],
            marker=config['marker'],
            label=model_name,
            zorder=5,
            edgecolors='black',
            linewidth=1.5
        )
        
        # Plot CI as error bar
        if b['ci_lower'] and b['ci_upper']:
            ax.errorbar(
                m['baseline'] * 100,
                m['lambda_crit'],
                yerr=[[m['lambda_crit'] - b['ci_lower']], [b['ci_upper'] - m['lambda_crit']]],
                fmt='none',
                c=config['color'],
                capsize=5,
                capthick=2,
                elinewidth=2,
                zorder=4
            )

# Fit trend line (excluding λ*=1.0 models)
valid_points = [(metrics[m]['baseline'], metrics[m]['lambda_crit']) 
                for m in metrics if metrics[m]['lambda_crit'] and metrics[m]['lambda_crit'] < 1.0]
if len(valid_points) >= 2:
    baselines = [p[0] * 100 for p in valid_points]
    lambdas = [p[1] for p in valid_points]
    z = np.polyfit(baselines, lambdas, 1)
    p = np.poly1d(z)
    x_line = np.linspace(min(baselines) - 5, max(baselines) + 5, 100)
    ax.plot(x_line, p(x_line), 'k--', linewidth=2, alpha=0.5, label=f'Trend (slope={z[0]:.4f})')

ax.set_xlabel('Baseline Accuracy (%)', fontsize=14, fontweight='bold')
ax.set_ylabel('λ* (Backfire Boundary)', fontsize=14, fontweight='bold')
ax.set_title('Scaling Law of Vulnerability:\nHigher Capability → Lower λ* → More Susceptible', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xlim(25, 100)
ax.set_ylim(0.3, 1.05)

# Add annotation for key finding
ax.annotate(
    'Claude 4 Sonnet:\nHighest capability (92.5%)\nLowest λ* (≈0.4)',
    xy=(92.5, 0.4), xytext=(70, 0.5),
    fontsize=10,
    arrowprops=dict(arrowstyle='->', color='purple', lw=2),
    bbox=dict(boxstyle='round', facecolor='lavender', alpha=0.8)
)

plt.tight_layout()
plt.savefig(f'{ANALYSIS_DIR}/figures/fig1_baseline_vs_lambda_crit.png', dpi=300, bbox_inches='tight')
plt.savefig(f'{ANALYSIS_DIR}/figures/fig1_baseline_vs_lambda_crit.pdf', bbox_inches='tight')
plt.show()
print(f"Saved: fig1_baseline_vs_lambda_crit.png/pdf")

In [ ]:
# Figure 2: All Collapse Curves

fig, ax = plt.subplots(figsize=(12, 8))

for model_name, data in all_data.items():
    m = metrics[model_name]
    config = data['config']
    
    lams = sorted(m['acc_by_lam'].keys())
    accs = [m['acc_by_lam'][l] * 100 for l in lams]
    
    ax.plot(
        lams, accs,
        marker=config['marker'],
        color=config['color'],
        linewidth=2.5,
        markersize=10,
        label=f"{model_name} (B={m['baseline']*100:.1f}%)"
    )
    
    # Add baseline as horizontal dashed line
    ax.axhline(y=m['baseline']*100, color=config['color'], linestyle=':', alpha=0.5)

ax.set_xlabel('Corruption Rate (λ)', fontsize=14, fontweight='bold')
ax.set_ylabel('Accuracy (%)', fontsize=14, fontweight='bold')
ax.set_title('CoT Collapse Curves Across Models', fontsize=14, fontweight='bold')
ax.legend(loc='lower left', fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0, 105)

# Add secondary x-axis for Alignment
ax2 = ax.twiny()
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])
ax2.set_xticklabels(['1.0', '0.8', '0.6', '0.4', '0.2', '0.0'])
ax2.set_xlabel('Alignment (A = 1 - λ)', fontsize=12)

plt.tight_layout()
plt.savefig(f'{ANALYSIS_DIR}/figures/fig2_all_collapse_curves.png', dpi=300, bbox_inches='tight')
plt.savefig(f'{ANALYSIS_DIR}/figures/fig2_all_collapse_curves.pdf', bbox_inches='tight')
plt.show()
print(f"Saved: fig2_all_collapse_curves.png/pdf")

In [ ]:
# Figure 3: Bootstrap Distribution of λ* for Claude 4 Sonnet

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (model_name, b) in enumerate(bootstrap_results.items()):
    if idx >= 6:
        break
    
    ax = axes[idx]
    
    if b['samples']:
        ax.hist(b['samples'], bins=30, color=all_data[model_name]['config']['color'], 
                alpha=0.7, edgecolor='black')
        
        # Add vertical lines for CI
        if b['ci_lower'] and b['ci_upper']:
            ax.axvline(b['point_estimate'], color='red', linewidth=2, label=f"λ*={b['point_estimate']:.3f}")
            ax.axvline(b['ci_lower'], color='red', linestyle='--', linewidth=1.5)
            ax.axvline(b['ci_upper'], color='red', linestyle='--', linewidth=1.5)
            ax.axvspan(b['ci_lower'], b['ci_upper'], alpha=0.2, color='red', label='95% CI')
    
    ax.set_title(f"{model_name}\nλ*={b['point_estimate']:.3f} [{b['ci_lower']:.3f}, {b['ci_upper']:.3f}]" 
                 if b['ci_lower'] else model_name, fontsize=11)
    ax.set_xlabel('λ*')
    ax.set_ylabel('Count')
    ax.legend(fontsize=8)

plt.suptitle('Bootstrap Distributions of λ* (n=1000)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{ANALYSIS_DIR}/figures/fig3_bootstrap_distributions.png', dpi=300, bbox_inches='tight')
plt.show()
print(f"Saved: fig3_bootstrap_distributions.png")

## 7. Summary Table for Publication

In [ ]:
# Create comprehensive summary table

summary_data = []

for model_name in sorted(metrics.keys(), key=lambda x: metrics[x]['baseline']):
    m = metrics[model_name]
    b = bootstrap_results[model_name]
    s = supplementary[model_name]
    
    row = {
        'Model': model_name,
        'N': m['n_problems'],
        'Baseline': f"{m['baseline']*100:.1f}%",
        'λ*': f"{m['lambda_crit']:.3f}" if m['lambda_crit'] else 'N/A',
        '95% CI': f"[{b['ci_lower']:.3f}, {b['ci_upper']:.3f}]" if b['ci_lower'] else 'N/A',
        'Δ@λ=1': f"{s['delta_at_1']*100:+.1f}%",
        'Harm Area': f"{s['harm_area']:.3f}",
        'Benefit Area': f"{s['benefit_area']:.3f}"
    }
    summary_data.append(row)

summary_df = pd.DataFrame(summary_data)

print('='*100)
print('TABLE 1: Summary of A3 Scaling Law Experiment Results')
print('='*100)
print(summary_df.to_string(index=False))

# Save as CSV
summary_df.to_csv(f'{ANALYSIS_DIR}/table1_summary.csv', index=False)
print(f"\nSaved: table1_summary.csv")

## 8. Save All Results

In [ ]:
# Compile all results into a single JSON

final_results = {
    'experiment': 'A3_Scaling_Law_Statistical_Analysis',
    'date': pd.Timestamp.now().strftime('%Y-%m-%d'),
    'n_models': len(metrics),
    'models': {}
}

for model_name in metrics.keys():
    m = metrics[model_name]
    b = bootstrap_results[model_name]
    s = supplementary[model_name]
    
    final_results['models'][model_name] = {
        'basic_metrics': {
            'n_problems': m['n_problems'],
            'baseline_accuracy': m['baseline'],
            'accuracy_by_lambda': {str(k): v for k, v in m['acc_by_lam'].items()},
            'lambda_crit_point': m['lambda_crit']
        },
        'bootstrap': {
            'lambda_crit_mean': b['boot_mean'],
            'lambda_crit_std': b['boot_std'],
            'ci_lower': b['ci_lower'],
            'ci_upper': b['ci_upper'],
            'n_valid_samples': b['n_valid_samples']
        },
        'supplementary': s
    }

save_json(final_results, f'{ANALYSIS_DIR}/a3_complete_analysis.json')
print(f"Saved: a3_complete_analysis.json")

print('\n' + '='*70)
print('ANALYSIS COMPLETE')
print('='*70)
print(f'\nAll outputs saved to: {ANALYSIS_DIR}')
print('\nFiles generated:')
print('  - figures/fig1_baseline_vs_lambda_crit.png/pdf')
print('  - figures/fig2_all_collapse_curves.png/pdf')
print('  - figures/fig3_bootstrap_distributions.png')
print('  - table1_summary.csv')
print('  - a3_complete_analysis.json')